In [34]:
import pandas as pd

In [35]:
# 加载附件7的数据
data = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件7.xlsx')

## 按品类分组，计算每个品类的销售总量和平均销售价格

In [36]:
# 按品类分组，计算销售总量和销售金额
grouped = data.groupby('分类编码')
category_sales = grouped['销量(千克)'].sum()
category_sales_value = grouped.apply(lambda x: (x['销量(千克)'] * x['销售单价(元/千克)']).sum())

# 计算平均销售价格
category_avg_price = category_sales_value / category_sales

# 创建包含结果的DataFrame
result_df = pd.DataFrame({
    '销售总量(千克)': category_sales,
    '平均销售价格(元/千克)': category_avg_price
})

# 重置索引，使结果更易于阅读
result_df.reset_index(inplace=True)

# 打印结果
print(result_df)

         分类编码    销售总量(千克)  平均销售价格(元/千克)
0  1011010101  198520.978      5.435545
1  1011010201   41766.451      8.996496
2  1011010402   40581.353      8.626866
3  1011010501   22431.782      8.520244
4  1011010504   91588.629      8.233915
5  1011010801   76086.725      8.143310


## 制定补货计划，考虑每个品类的库存水平、损耗率和需求数据。

要制定补货计划，考虑每个品类的库存水平、损耗率和需求数据，您可以采用以下步骤：

1. **计算当前库存水平**：
   - 如果您有之前的库存数据，可以使用它们来计算当前库存水平。
   - 如果没有之前的库存数据，可以假设初始库存为零，然后根据销售和补货情况来更新库存。

2. **考虑损耗率**：
   - 使用附件5中的损耗率数据，估计每个品类的损耗率。
   - 计算每个品类的损耗量，将其减去销售量以获得净需求。

3. **考虑需求数据**：
   - 使用附件7中的销售数据，估计每个品类的需求。
   - 可以根据历史销售数据来预测未来一周的需求。

4. **制定补货计划**：
   - 将当前库存、损耗量和需求数据结合在一起，以制定补货计划。
   - 补货计划可以包括以下元素：
     - 计划购买数量：根据净需求（需求减去库存和预计损耗）确定需要购买的数量。
     - 补货时间：确定何时下订单以确保及时供应。
     - 补货频率：确定是每日、每周还是其他频率进行补货。
     - 补货量：根据需求和供应情况确定每次补货的数量。

5. **库存管理和调整**：
   - 定期监控库存水平、损耗情况和实际销售数据。
   - 根据实际情况进行库存调整和补货计划的更新。

6. **优化补货计划**：
   - 考虑使用优化算法来制定最佳补货计划，以最小化成本或最大化利润。
   - 在优化模型中，可以考虑成本、供应延迟、订购量限制等因素。

需要根据您的具体情况和数据来制定详细的补货计划。此外，建议使用库存管理系统或专业的供应链规划工具来更有效地管理和优化库存和补货过程。这些步骤将帮助您更好地管理蔬菜品类的库存，减少损耗，同时确保满足市场需求。

### 计算当前库存水平

In [37]:
# 按品类分组，计算每个品类的销售总量和补货总量
grouped = data.groupby('分类编码')
category_sales = grouped['销量(千克)'].sum()
category_replenishment = grouped['销量(千克)'].sum()  # 假设补货量和销售量相等

# 计算当前库存水平
current_inventory = category_replenishment.sum() - category_sales.sum()

# 打印每个品类的销售总量和补货总量
print("每个品类的销售总量：")
print(category_sales)
print("\n每个品类的补货总量：")
print(category_replenishment)

# 打印当前库存水平
print("\n当前库存水平：", current_inventory)

每个品类的销售总量：
分类编码
1011010101    198520.978
1011010201     41766.451
1011010402     40581.353
1011010501     22431.782
1011010504     91588.629
1011010801     76086.725
Name: 销量(千克), dtype: float64

每个品类的补货总量：
分类编码
1011010101    198520.978
1011010201     41766.451
1011010402     40581.353
1011010501     22431.782
1011010504     91588.629
1011010801     76086.725
Name: 销量(千克), dtype: float64

当前库存水平： 0.0


### 考虑损耗率：


#### 使用附件5中的损耗率数据，估计每个品类的损耗率。

In [38]:
# 加载附件5中的损耗率数据
loss_data = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件5.xlsx')

In [39]:
# 加载附件7的数据（销售数据）
sales_data = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件7.xlsx')

In [40]:
# 合并销售数据和损耗率数据，根据单品编码匹配
merged_data = pd.merge(sales_data, loss_data, on='单品编码', how='left')

# 按品类分组，计算每个品类的平均损耗率
grouped = merged_data.groupby('分类编码')
category_avg_loss_rate = grouped['损耗率(%)'].mean()

# 打印每个品类的平均损耗率
print("每个品类的平均损耗率：")
print(category_avg_loss_rate)

每个品类的平均损耗率：
分类编码
1011010101    12.733681
1011010201    10.664638
1011010402     8.911259
1011010501     6.411989
1011010504     7.755571
1011010801     8.884176
Name: 损耗率(%), dtype: float64


#### 	计算每个品类的损耗量，将其减去销售量以获得净需求。

In [41]:
# 计算每个品类的销售总量和损耗量
grouped = merged_data.groupby('分类编码')
category_sales = grouped['销量(千克)'].sum()
category_loss_rate = grouped['损耗率(%)'].mean()
category_loss = (category_sales * category_loss_rate / 100)

# 计算净需求（销售量减去损耗量）
category_net_demand = category_sales - category_loss

# 打印每个品类的销售总量、损耗量和净需求
result_df = pd.DataFrame({
    '销售总量(千克)': category_sales,
    '损耗量(千克)': category_loss,
    '净需求(千克)': category_net_demand
})

result_df.reset_index(inplace=True)
print(result_df)

         分类编码    销售总量(千克)       损耗量(千克)        净需求(千克)
0  1011010101  198520.978  25279.028749  173241.949251
1  1011010201   41766.451   4454.240990   37312.210010
2  1011010402   40581.353   3616.309495   36965.043505
3  1011010501   22431.782   1438.323434   20993.458566
4  1011010504   91588.629   7103.221345   84485.407655
5  1011010801   76086.725   6759.678671   69327.046329


### 考虑需求数据

#### 使用附件7中的销售数据，估计每个品类的需求。

使用2020年7月1日到2023年6月30日的当前库存水平current_inventory，每个品类的平均损耗率：category_avg_loss_rate，每个品类的净需求：result_df等数据估计每个品类2023年7月1日到7日需求

In [42]:
# 加载附件7的数据
data = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件7.xlsx')

In [43]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [44]:
# 加载历史销售数据
historical_sales_data = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件7.xlsx')

# 加载当前库存水平
current_inventory = 0.0  # 请将此值更新为实际的当前库存水平

# 加载每个品类的平均损耗率数据
category_avg_loss_rate = pd.Series({
    '1011010101': 12.733681,
    '1011010201': 10.664638,
    '1011010402': 8.911259,
    '1011010501': 6.411989,
    '1011010504': 7.755571,
    '1011010801': 8.884176
})


数据预处理：将历史销售数据转换为时间序列，以便应用ARIMA模型。这包括日期列的设置、聚合销售数据到适当的时间间隔（例如，每日），并将其分为各个品类。

拟合ARIMA模型：为每个品类的时间序列拟合ARIMA模型，并选择适当的模型参数（如p、d和q值）。

预测：使用已拟合的ARIMA模型，对每个品类的需求进行2023年7月1日到7日的预测。

In [45]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

# 加载历史销售数据
sales_data = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件7.xlsx')

# 将销售日期列转换为日期时间类型
sales_data['销售日期'] = pd.to_datetime(sales_data['销售日期'])

# 按分类编码分组
grouped = sales_data.groupby('分类编码')

# 创建一个DataFrame来存储预测结果
forecast_results = pd.DataFrame(columns=['分类编码', '日期', '预测需求'])

# 定义预测的日期范围（2023年7月1日到7日）
forecast_dates = pd.date_range(start='2023-07-01', end='2023-07-07')

# 循环遍历每个品类
for category_code, group_data in grouped:
    # 创建该品类的时间序列数据，按日期汇总销售量
    category_time_series = group_data.groupby('销售日期')['销量(千克)'].sum().resample('D').sum()
    
    # 拟合ARIMA模型，这里选择ARIMA(1,1,1)模型作为示例
    model = ARIMA(category_time_series, order=(1, 1, 1))
    model_fit = model.fit()
    
    # 预测未来七天的需求
    forecast_values = model_fit.forecast(steps=7)
    
    # 创建DataFrame存储预测结果
    forecast_df = pd.DataFrame({
        '分类编码': [category_code] * 7,
        '日期': forecast_dates,
        '预测需求': forecast_values
    })
    
    # 将预测结果添加到总的预测结果DataFrame中
    forecast_results = pd.concat([forecast_results, forecast_df])

# 打印预测结果
print(forecast_results)


F:\language\ANACONDA\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                  分类编码                   日期        预测需求
2023-07-01  1011010101  2023-07-01 00:00:00  139.954275
2023-07-02  1011010101  2023-07-02 00:00:00  142.601358
2023-07-03  1011010101  2023-07-03 00:00:00  143.339697
2023-07-04  1011010101  2023-07-04 00:00:00  143.545639
2023-07-05  1011010101  2023-07-05 00:00:00  143.603081
2023-07-06  1011010101  2023-07-06 00:00:00  143.619104
2023-07-07  1011010101  2023-07-07 00:00:00  143.623573
2023-07-01  1011010201  2023-07-01 00:00:00   22.976202
2023-07-02  1011010201  2023-07-02 00:00:00   20.685084
2023-07-03  1011010201  2023-07-03 00:00:00   19.657999
2023-07-04  1011010201  2023-07-04 00:00:00   19.197568
2023-07-05  1011010201  2023-07-05 00:00:00   18.991161
2023-07-06  1011010201  2023-07-06 00:00:00   18.898631
2023-07-07  1011010201  2023-07-07 00:00:00   18.857151
2023-07-01  1011010402  2023-07-01 00:00:00    18.39016
2023-07-02  1011010402  2023-07-02 00:00:00   18.005223
2023-07-03  1011010402  2023-07-03 00:00:00   17

## 制定定价策略示例

In [67]:
# 假设每个品类的成本
cost_per_category = {
    '1011010101': 4.0,  # 计算平均成本
    '1011010201': 4.0,
    '1011010402': 6.0,
    '1011010501': 3.0,
    '1011010504': 7.0,
    '1011010801': 5.5,
}

# 假设的市场需求强度（0到1之间，1表示最高需求）
demand_intensity_per_category = {
    '1011010101': 0.8,
    '1011010201': 0.6,
    '1011010402': 0.7,
    '1011010501': 0.9,
    '1011010504': 0.8,
    '1011010801': 0.7,
}

# 定价策略：基于成本加成和需求强度
pricing_strategy = {}
for category_code in cost_per_category:
    cost = cost_per_category[category_code]
    demand_intensity = demand_intensity_per_category[category_code]
    
    # 假设的加成率，可以根据需求强度进行调整
    markup_rate = 0.2  # 20%的加成率
    
    # 根据成本和加成率计算价格
    price = cost * (1 + markup_rate)
    
    # 根据需求强度调整价格
    price *= (1 + demand_intensity)
    
    pricing_strategy[category_code] = price

# 打印每个品类的定价策略
print("每个品类的定价策略：")
print(pricing_strategy)


每个品类的定价策略：
{'1011010101': 10.8, '1011010201': 7.68, '1011010402': 12.239999999999998, '1011010501': 6.839999999999999, '1011010504': 15.120000000000001, '1011010801': 11.219999999999999}


## 计算每个品类的平均成本和市场需求强度

In [69]:
import pandas as pd

# 加载附件7的数据
data7 = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件7.xlsx')

# 加载附件3的数据
data3 = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件3.xlsx')

# 合并数据，根据销售日期和单品编码对应
merged_data = pd.merge(data7, data3[['销售日期', '单品编码', '批发价格(元/千克)']], on=['销售日期', '单品编码'], how='left')

# 导出合并后的数据到附件8
merged_data.to_excel(r'C:\Users\Desktop\附件8.xlsx', index=False)


In [71]:
import pandas as pd

# 加载附件8的数据
data8 = pd.read_excel(r'C:\Users\32335\MachineLearning\TC\附件8.xlsx')

# 计算每个品类的平均成本
average_cost_per_category = data8.groupby('分类编码')['批发价格(元/千克)'].mean()

# 计算市场需求强度
total_sales = data8['销量(千克)'].sum()
market_demand_intensity_per_category = data8.groupby('分类编码')['销量(千克)'].sum() / total_sales

# 创建包含结果的DataFrame
result_df = pd.DataFrame({
    '分类编码': average_cost_per_category.index,
    '平均成本(元/千克)': average_cost_per_category.values,
    '市场需求强度': market_demand_intensity_per_category.values
})

# 打印每个品类的平均成本和市场需求强度
print("每个品类的平均成本和市场需求强度：")
print(result_df)


每个品类的平均成本和市场需求强度：
         分类编码  平均成本(元/千克)    市场需求强度
0  1011010101    3.841342  0.421510
1  1011010201    5.961771  0.088681
2  1011010402    6.622156  0.086164
3  1011010501    5.547849  0.047628
4  1011010504    6.744188  0.194466
5  1011010801    7.704686  0.161551


## 根据个品类的平均成本和市场需求强度制定定价策略

In [72]:
# 假设每个品类的成本
cost_per_category = {
    '1011010101': 3.841342,  # 计算平均成本
    '1011010201': 5.961771,
    '1011010402': 6.622156,
    '1011010501': 5.547849,
    '1011010504': 6.744188,
    '1011010801': 7.704686,
}

# 假设的市场需求强度（0到1之间，1表示最高需求）
demand_intensity_per_category = {
    '1011010101': 0.421510,
    '1011010201': 0.088681,
    '1011010402': 0.086164,
    '1011010501': 0.047628,
    '1011010504': 0.194466,
    '1011010801': 0.161551,
}

# 定价策略：基于成本加成和需求强度
pricing_strategy = {}
for category_code in cost_per_category:
    cost = cost_per_category[category_code]
    demand_intensity = demand_intensity_per_category[category_code]
    
    # 假设的加成率，可以根据需求强度进行调整
    markup_rate = 0.2  # 20%的加成率
    
    # 根据成本和加成率计算价格
    price = cost * (1 + markup_rate)
    
    # 根据需求强度调整价格
    price *= (1 + demand_intensity)
    
    pricing_strategy[category_code] = price

# 打印每个品类的定价策略
print("每个品类的定价策略：")
print(pricing_strategy)


每个品类的定价策略：
{'1011010101': 6.5526072797040005, '1011010201': 7.788560176861199, '1011010402': 8.6312969395008, '1011010501': 6.974498342606401, '1011010504': 9.6668439163296, '1011010801': 10.739262873583199}
